In [1]:
import pandas as pd
import numpy as np 
import time 
from keras.models import load_model

test_file = 'data/pems_test.csv'
df = pd.read_csv(test_file, encoding='utf-8').fillna(0)
df = np.array(df)
lstm = load_model("TrafficPrediction/lstm_model.h5")
scale_ = 0.005076142131979695

Using TensorFlow backend.


In [2]:
n = df.shape[0]
lag = 12
for i in range(0, n):
    data = df[i:i+lag]
    flows = data[:, 1]
    # Transform the data for prediction
    flows = np.asarray(flows)
    flows = np.multiply(flows, scale_)
    print(flows)
    flows = np.reshape(flows, (1, lag, 1))
    
    # Make the prediction
    preds = lstm.predict(flows)
    
    # Inverse transform
    preds = np.divide(preds, scale_)
    print("Predictions : {}".format(preds))
    
    time.sleep(0.25)

[0.08121827411167512 0.050761421319796954 0.055837563451776644
 0.055837563451776644 0.03045685279187817 0.06598984771573603
 0.035532994923857864 0.01015228426395939 0.03045685279187817
 0.035532994923857864 0.02030456852791878 0.035532994923857864]
Predictions : [[0.]]
[0.050761421319796954 0.055837563451776644 0.055837563451776644
 0.03045685279187817 0.06598984771573603 0.035532994923857864
 0.01015228426395939 0.03045685279187817 0.035532994923857864
 0.02030456852791878 0.035532994923857864 0.06091370558375634]
Predictions : [[1.715934]]
[0.055837563451776644 0.055837563451776644 0.03045685279187817
 0.06598984771573603 0.035532994923857864 0.01015228426395939
 0.03045685279187817 0.035532994923857864 0.02030456852791878
 0.035532994923857864 0.06091370558375634 0.025380710659898477]
Predictions : [[1.3144711]]
[0.055837563451776644 0.03045685279187817 0.06598984771573603
 0.035532994923857864 0.01015228426395939 0.03045685279187817
 0.035532994923857864 0.02030456852791878 0.035

[0.025380710659898477 0.02030456852791878 0.02030456852791878
 0.03045685279187817 0.005076142131979695 0.025380710659898477
 0.03045685279187817 0.015228426395939085 0.015228426395939085
 0.025380710659898477 0.02030456852791878 0.01015228426395939]
Predictions : [[0.]]
[0.02030456852791878 0.02030456852791878 0.03045685279187817
 0.005076142131979695 0.025380710659898477 0.03045685279187817
 0.015228426395939085 0.015228426395939085 0.025380710659898477
 0.02030456852791878 0.01015228426395939 0.04060913705583756]
Predictions : [[0.]]
[0.02030456852791878 0.03045685279187817 0.005076142131979695
 0.025380710659898477 0.03045685279187817 0.015228426395939085
 0.015228426395939085 0.025380710659898477 0.02030456852791878
 0.01015228426395939 0.04060913705583756 0.04060913705583756]
Predictions : [[0.]]
[0.03045685279187817 0.005076142131979695 0.025380710659898477
 0.03045685279187817 0.015228426395939085 0.015228426395939085
 0.025380710659898477 0.02030456852791878 0.0101522842639593

[0.21319796954314718 0.24365482233502536 0.23350253807106597
 0.3096446700507614 0.2893401015228426 0.29441624365482233
 0.33502538071065985 0.34010152284263956 0.532994923857868
 0.45177664974619286 0.47208121827411165 0.45177664974619286]
Predictions : [[98.82561]]
[0.24365482233502536 0.23350253807106597 0.3096446700507614
 0.2893401015228426 0.29441624365482233 0.33502538071065985
 0.34010152284263956 0.532994923857868 0.45177664974619286
 0.47208121827411165 0.45177664974619286 0.5177664974619289]
Predictions : [[102.45332]]
[0.23350253807106597 0.3096446700507614 0.2893401015228426
 0.29441624365482233 0.33502538071065985 0.34010152284263956
 0.532994923857868 0.45177664974619286 0.47208121827411165
 0.45177664974619286 0.5177664974619289 0.5431472081218274]
Predictions : [[107.02157]]
[0.3096446700507614 0.2893401015228426 0.29441624365482233
 0.33502538071065985 0.34010152284263956 0.532994923857868
 0.45177664974619286 0.47208121827411165 0.45177664974619286
 0.517766497461928

KeyboardInterrupt: 